In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import jax
import flax

import numpy as np

import configs
import models
import data
import train
import utils

2024-01-30 14:22:51.875036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 14:22:51.875085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 14:22:51.876411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 14:22:52.931331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
task_config = configs.TaskConfig()
task_config.lora_compress = True
task_config.lora_rank = 8

In [6]:
is_regression = task_config.finetune_task_name == "stsb"

# Model
pretrain_model = models.create_pretrain_model_from_config(
    task_config, num_labels=data.task_to_num_labels[task_config.finetune_task_name]
)

learning_rate_fn = train.create_learning_rate_fn(
    task_config.num_train_steps,
    task_config.num_warmup_steps,
    task_config.learning_rate,
    task_config.decay_ratio,
)

train_step, eval_step = train.create_train_eval_step_fns(learning_rate_fn)

model_state = train.create_model_state(
    model=pretrain_model,
    is_regression=is_regression,
)

2024-01-30 14:23:07.540179: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Some weights of the model checkpoint at bert-base-cased were not used when initializing FlaxBertForSequenceClassification: {('cls', 'predictions', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias')}
- This IS expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

In [7]:
seed = 0
train_dataset, eval_dataset = data.load_dataset_from_config(task_config, seed)

rng = jax.random.PRNGKey(seed)

lora_rng, rng = jax.random.split(rng)
uncompressed_lora_state = train.create_lora_train_state(
    task_config,
    pretrain_model.params,  # type: ignore
    learning_rate_fn=learning_rate_fn,
    lora_rng=lora_rng,
)

experiment_path = utils.get_experiment_path(task_config, seed)
print(experiment_path)

dropout_rng, input_rng, rng = jax.random.split(rng, 3)
train_iterator = data.create_train_iterator(
    input_rng, train_dataset, task_config.train_batch_size
)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

/home/ubuntu/deep-lora-transformers/experiments/stsb_lora_type=only-query-value_depth=3_rank=8_compress_lr=0.0001_steps=200_seed=0
